In [40]:
import ("github.com/e-XpertSolutions/go-iforest/iforest"
		"bufio"
		"encoding/csv"
		"strconv"
		"github.com/janpfeifer/gonb/gonbui"
		"gonum.org/v1/plot/plotutil"
		"gonum.org/v1/plot"
		"gonum.org/v1/plot/plotter"
		"gonum.org/v1/plot/vg"
		"gonum.org/v1/plot/vg/draw"
		"os/exec"
		"github.com/janpfeifer/gonb/cache"

	)

	

# UTILITIES

In [41]:


//load file Data and Return [][]float64 array
func loadData(myfile string) [][]float64 {

	l:=false
file, err := os.Open(myfile)
	if err != nil {
		fmt.Println("Error opening file:", err)
		return nil
	}
	defer file.Close()

	reader := csv.NewReader(file)
	records, err := reader.ReadAll()
	if err != nil {
		fmt.Println("Error reading CSV:", err)
		return nil
	}
	
	inputData := make([][]float64, len(records))
	for i, row := range records {

		inputData[i] = make([]float64, len(row))
		for j, value := range row {
			inputData[i][j], err = strconv.ParseFloat(value, 64)
			if err != nil {
				fmt.Printf("Error converting to float: line %d",i)
				l=true
				break	
			}
		}
		if l == true{
			break
		}
	}
	return inputData
}
// Compare algorithm results with data extracted from original file
func compareResults(labels []int){

	count :=0
	line :=0
	// Open the CSV file
	filePath := "labels.csv"

	// Open the file
	file, err := os.Open(filePath)
	if err != nil {
		fmt.Println("Error opening file:", err)
		return
	}
	defer file.Close()

	// Create a new scanner
	scanner := bufio.NewScanner(file)

	// Read and process each line
	for scanner.Scan() {
		if scanner.Text()== "benign" && labels[line] ==0{
			count++
			line++
		}else if scanner.Text()== "malignant" && labels[line] ==1{
			count++
			line++
		}else if scanner.Text()== "outlier" && labels[line] ==1{
			count++
			line++
		}else{
			line++
		}
	}

	accuracy := float64(count) / float64(line) * 100.0
	fmt.Printf("%d lines of which %d acurrate\n",line,count)

    fmt.Printf("Percentage of Accuracy is %.2f%%\n", accuracy)
	
}
// call command line executable
func prepareDataset(dataset string){
	cmd :=exec.Command("./extractDataset.sh", dataset)
	cmd.Run()
}



# Iso Forest Algorithm

In [42]:
//Isolation Forest algorithm

func isoForestTrain_Test(dataset string, treesNumber int, subsampleSize int, outliers float64) *iforest.Forest{
//model initialization
forest := iforest.NewForest(treesNumber, subsampleSize, outliers)

prepareDataset(dataset)
//training stage - creating trees
inputData := loadData("dataset.csv")
forest.Train(inputData)

//testing stage - finding anomalies 
forest.Test(inputData)

threshold := forest.AnomalyBound
labels := forest.Labels
//fmt.Println("labels is", labels)
fmt.Println("threshold is",threshold)


compareResults(labels)

return forest;
}


# Predict function

In [43]:
func predict(forest * iforest.Forest, dataset string){

	prepareDataset(dataset)
	newData := loadData("dataset.csv")
	labels1,_ ,_ := forest.Predict(newData)
	// compare results with original file
	compareResults(labels1)
	}

In [46]:
var forest = cache.Cache("my_forest", func() * iforest.Forest {return isoForestTrain_Test("2020.06.22.csv",100,256,0.14)})

10.244770920116851
threshold is 0.03719914579244621
4204081 lines of which 3635765 acurrate
Percentage of Accuracy is 86.48%


In [45]:
%%
// reset 
cache.ResetKey("my_forest")

In [47]:
%%
predict(forest,"2020.06.21.csv")

814816 lines of which 402914 acurrate
Percentage of Accuracy is 49.45%
